# 제 4 장 데이터 수집과 정리
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumMyeongjo')
rc('axes', unicode_minus=False)

___

## 4.2 __Open API 이용__

In [ ]:
import requests as req
import json
import pandas as pd

(1) 한국은행 Open API

* 홈페이지 : https://ecos.bok.or.kr/api/#/

* Open API 인증키 신청 후 [My Page]에서 확인

In [ ]:
BOK_KEY = 'N1VG2759HK9V3HDMUG1U'

* 이용 예시

In [ ]:
uri = 'http://ecos.bok.or.kr/api/StatisticTableList/N1VG2759HK9V3HDMUG1U/json/kr/1/10/102Y004'
resp = req.get(uri)

In [ ]:
resp.text

In [ ]:
res1 = json.loads(resp.text)

In [ ]:
res1

In [ ]:
res1['StatisticTableList']

In [ ]:
res1['StatisticTableList']['row']

In [ ]:
df1 = pd.json_normalize(res1['StatisticTableList']['row'])

In [ ]:
df1

* 통계 용어 사전 활용

In [ ]:
uri = 'http://ecos.bok.or.kr/api/StatisticWord/N1VG2759HK9V3HDMUG1U/json/kr/1/10/'

In [ ]:
word = '소비자동향지수'

In [ ]:
resp = req.get(uri+word)

In [ ]:
resp.status_code

In [ ]:
resp.text

In [ ]:
res2 = json.loads(resp.text)

In [ ]:
df2 = pd.json_normalize(res2['StatisticWord']['row'])

In [ ]:
df2

* 최근 3년간의 소비자물가지수(분기별 총지수) 가져오기

In [ ]:
uri_fmt = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/json/kr/1/1000/{}/{}/{}/{}/{}'

In [ ]:
통계코드,작성주기,시작시점,종료시점,항목코드= '901Y009','M','202101','202312','0'

In [ ]:
uri = uri_fmt.format(BOK_KEY,통계코드,작성주기,시작시점,종료시점,항목코드)

In [ ]:
resp = req.get(uri)

In [ ]:
resp

In [ ]:
tmp = json.loads(resp.text)

In [ ]:
res = pd.json_normalize(tmp['StatisticSearch']['row'])

In [ ]:
res


(2) 네이버 검색 API

* 애플리케이션 등록 후 API 인증 정보

In [ ]:
CLT_ID = 'Q8Pw7CUP8kMmHB51FSGA'
CLT_SCR = 'wguPXntry8'

* 뉴스 검색

In [ ]:
API_URL = 'https://openapi.naver.com/v1/search/book.json'
params = {
    'query' : '방탄소년단',
    'display' : 100,
    'start' : 1,
    'sort' : 'date'
}
headers = {
    'X-Naver-Client-Id' : CLT_ID,
    'X-Naver-Client-Secret' : CLT_SCR
}

In [ ]:
resp = req.get(API_URL, params=params, headers=headers)

In [ ]:
resp

In [ ]:
resp.status_code

In [ ]:
cnt = resp.json()

In [ ]:
cnt

In [ ]:
cnt['items']

In [ ]:
df1 = pd.json_normalize(cnt['items'])

In [ ]:
df1 = pd.json_normalize((resp.json())['items'])

In [ ]:
df1.head()

In [ ]:
df1.dtypes

(3) kakao Karlo API

* 카카오 앱 사용정보

In [ ]:
# [내 애플리케이션] > [앱 키] 에서 확인한 REST API 키 값 입력
KAKAO_REST_API_KEY = '81ea39b104eb74d11bf37cfb34b10915'

* 주소를 좌표로 변환하기

In [ ]:
address = '경기도 수원시 영통구 광교산로 154-42'

- 네이버와 비슷한 형식으로 사용

In [ ]:
API_URL = 'https://dapi.kakao.com/v2/local/search/address.json'
params = {
    'query' : address
}
headers = {
    'Authorization' : 'KakaoAK ' + KAKAO_REST_API_KEY
}

In [ ]:
resp = req.get(API_URL, params=params, headers=headers)

In [ ]:
# 응답 결과 확인
resp.json()

In [ ]:
cnt=resp.json()

In [ ]:
cnt
#cnt['documents']
#cnt['documents'][0]
#cnt['documents'][0]['address']

In [ ]:
# 위도
cnt['documents'][0]['address']['y']

In [ ]:
# 경도
cnt['documents'][0]['address']['x']